### System libraries & setup

In [1]:
import chromadb
import os
import nest_asyncio
from dotenv import load_dotenv

In [4]:
nest_asyncio.apply()
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Llama Index libraries

In [5]:
from llama_index.packs.raptor import RaptorRetriever
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.query_engine import RetrieverQueryEngine

### Langchain libraries

In [6]:
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings as Embeddings

### User's query

In [7]:
query_raptor = "What did the March Hare offer to Alice at A Mad Tea-Party?"

### RAPTOR RAG

In [8]:
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [9]:
client_raptor = chromadb.PersistentClient(path="./raptor_db")
collection_raptor = client_raptor.get_collection("docs")
vector_store_raptor = ChromaVectorStore(chroma_collection=collection_raptor)

In [10]:
retriever_raptor = RaptorRetriever(
    [],
    vector_store=vector_store_raptor,  
    similarity_top_k=3,  
    mode="collapsed",  
)

In [11]:
query_engine = RetrieverQueryEngine.from_args(
    retriever_raptor, llm=OpenAI(model="gpt-3.5-turbo", temperature=0)
)

In [12]:
response = query_engine.query(query_raptor)

### Answer from RAPTOR RAG

In [13]:
print(response)

The March Hare offered Alice wine at A Mad Tea-Party.


### Context RAG

In [14]:
embeddings = Embeddings(model="text-embedding-3-small")

In [15]:
query_semantic = str(response)

In [16]:
client_semantic = chromadb.PersistentClient(path="./semantic_db")
collection = client_semantic.get_or_create_collection("docs")

In [17]:
db_chroma = Chroma(
    client=client_semantic ,
    collection_name="docs",
    embedding_function=embeddings,
)

In [18]:
chunks = db_chroma.similarity_search(query_semantic, k=3)

In [19]:
texts = ''.join(str(i) for i in chunks)

### Similar semantic chunks

In [20]:
print(texts)

page_content='While she was looking at the place\nwhere it had been, it suddenly appeared again. `By-the-bye, what became of the baby?\' said the Cat. `I\'d\nnearly forgotten to ask.\'\n\n  `It turned into a pig,\' Alice quietly said, just as if it had\ncome back in a natural way. `I thought it would,\' said the Cat, and vanished again. Alice waited a little, half expecting to see it again, but it\ndid not appear, and after a minute or two she walked on in the\ndirection in which the March Hare was said to live. `I\'ve seen\nhatters before,\' she said to herself; `the March Hare will be\nmuch the most interesting, and perhaps as this is May it won\'t be\nraving mad--at least not so mad as it was in March.\'  As she said\nthis, she looked up, and there was the Cat again, sitting on a\nbranch of a tree. `Did you say pig, or fig?\' said the Cat. `I said pig,\' replied Alice; `and I wish you wouldn\'t keep\nappearing and vanishing so suddenly:  you make one quite giddy.\'\n\n  `All right,\

### RAG agent

In [21]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [22]:
prompt_template = f"""
Your goal is to incorporate a provided best answer and then enrich it with additional relevant information from a given context. Follow these steps carefully:

1. First, you will be presented with a user's query. Read it carefully to understand the information being sought.

2. Next, you will be given a "best answer" to the query. This answer should form the foundation of your response.

3. Finally, you will be provided with additional context. This context may contain supplementary information that can be used to expand upon the best answer.

Here is the user's query:
<user_query>
{query_raptor}
</user_query>

Here is the best answer to the query:
<best_answer>
{query_semantic} 
</best_answer>

Begin your response by incorporating the information from the best answer. Paraphrase and restructure the content as needed to fit naturally into your response, but ensure that all key points from the best answer are included.

Now, consider the following additional context:
<context>
{texts}
</context>

After incorporating the best answer, enrich and expand your response using relevant information from the context. Look for details that:
- Provide additional explanation or clarification of concepts mentioned in the best answer
- Offer relevant examples or case studies
- Present related facts or statistics
- Introduce complementary or contrasting viewpoints
- Address potential follow-up questions the user might have

Integrate this information seamlessly into your response, ensuring a logical flow of ideas. Be sure to only include information that is directly relevant to the user's query and the best answer.

Remember to maintain a coherent and engaging writing style throughout your response. Aim for clarity and conciseness while providing a thorough answer to the user's query.
"""

In [23]:
messages = [
    ("system", "You are an AI assistant tasked with providing comprehensive and informative responses to user queries."),
    ("human", prompt_template),
]

In [24]:
answer = llm.invoke(messages)

### Final RAG answer

In [25]:
print(answer.content)

In Lewis Carroll's "Alice's Adventures in Wonderland," during the famous Mad Tea-Party scene, the March Hare offers Alice some wine. However, when Alice looks around the table, she finds that there is no wine present. The March Hare then admits, "There isn't any," which leads Alice to remark that it was not very civil of him to offer something that did not exist. The March Hare retorts that it was not very civil of Alice to sit down without being invited.

This interaction is a classic example of the whimsical and nonsensical nature of the characters and their dialogue in Wonderland. The tea party itself is a chaotic and surreal event, with the March Hare, the Hatter, and the Dormouse engaging in bizarre conversations and behavior. For instance, the Hatter asks Alice a riddle, "Why is a raven like a writing-desk?" but neither he nor the March Hare knows the answer, highlighting the absurdity of their world.

The setting of the tea party is also quite peculiar. The table is set out unde

### QUERIES

In [26]:
query_raptor = "What did the March Hare offer to Alice at A Mad Tea-Party?"

In [27]:
query_raptor = "What did Mad Hatter say to Alice at Mad Tea Party?"

In [28]:
query_raptor = "When did Cheshire Cat meet the Queen?"

In [29]:
query_raptor = "Who wakes up Alice at the end of the story?"

In [30]:
query_raptor = "Accurately describe the character of the Cheshire Cat"

In [31]:
query_raptor = "What is the role of the Dodo?"